In [7]:
# Establish a connection to the PostgreSQL database
import psycopg, os
print('Connecting to the PostgreSQL database...')
conn = psycopg.connect(
    host="localhost",
    port='5432',
    dbname="my_db",
    user="postgres",
    password="123")

Connecting to the PostgreSQL database...


In [8]:
# Create a cursor object using the connection
cur = conn.cursor()

In [9]:
print('PostgreSQL database version:')
cur.execute('SELECT version()')
db_version = cur.fetchone()
print(db_version)

PostgreSQL database version:
('PostgreSQL 16.2 (Debian 16.2-1.pgdg120+2) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)


In [10]:
# Define the SQL command to create the 'ceos' table
create_ceo_table_Cmd = """CREATE TABLE ceos (
            ceo_id SERIAL PRIMARY KEY,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            year_of_birth INTEGER NOT NULL,
            place_of_birth TEXT NOT NULL,
            start_year INTEGER NOT NULL
);
"""
cur.execute(create_ceo_table_Cmd)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=my_db) at 0x27c007d2720>

In [11]:
# Define the SQL command to create the 'companies' table
create_companies_table_Cmd = """CREATE TABLE companies(
    company_id SERIAL PRIMARY KEY,
    name TEXT NOT NULL,
    headquarters_city TEXT NOT NULL,
    founding_year INTEGER NOT NULL,
    ceo_id INTEGER REFERENCES ceos(ceo_id)
);
"""
cur.execute(create_companies_table_Cmd)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=my_db) at 0x27c007d2720>

In [12]:
# Define the SQL command to create the 'locations' table
create_LOCATIONS_table_Cmd = """CREATE TABLE Locations (
    location_id SERIAL PRIMARY KEY,
    city TEXT NOT NULL,
    state TEXT,
    country TEXT NOT NULL
);
"""
cur.execute(create_LOCATIONS_table_Cmd)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=my_db) at 0x27c007d2720>

In [22]:
# Inserting values into the 'ceos' table
insert_records_ceos = """insert into ceos (first_name, last_name, year_of_birth, place_of_birth, start_year)
VALUES ('Tim', 'Cook', 1960, 'Mobile, AL, USA', 2011),
('Arvind', 'Krishna', 1962, 'West Godavari, Andhra-Pradesh, India', 2020),
('Andy', 'Jassy', 1968, 'Scarsdale, NY, USA', 2021),
('Mary', 'Barra', 1961, 'Royal Oak, MI, USA', 2014),
('Kenichiro', 'Yoshida', 1959, 'Kumamoto, Japan', 2018),
('Daniel', 'Zhang', 1972, 'Shanghai, China', 2015),
('François-Henri', 'Pinault', 1962, 'Rennes, Illes-et-Villaine, France', 2005),
('Lee', 'Jae-yong', 1968, 'Seoul, South Korea', 2022),
('Doug', 'McMillon', 1966, 'Memphis, TN, USA', 2014)
"""
cur.execute(insert_records_ceos)

# Inserting values into the 'companies' table
insert_records_companies = """insert into companies (name, headquarters_city, founding_year, ceo_id)
VALUES ('Apple', 'Cupertino, CA', 1976, 1),
('IBM', 'Armonk, NY', 1911, 2),
('Amazon', 'Seattle, WA', 1994, 3),
('General Motors', 'Detroit, MI', 1908, 4),
('Sony Corporation', 'Tokyo', 1946, 5),
('Alibaba Group', 'Hangzhou, Zhejiang', 1999, 6),
('Kering SA', 'Paris', 1963, 7),
('Samsung Electronics', 'Suwon, Gyeonggi-do', 1969, 8),
('Walmart', 'Bentonville, Arkansas', 1962, 9)
"""
cur.execute(insert_records_companies)

# Inserting values into the 'locations' table
insert_records_locations = """insert into locations(city, country)
VALUES ('Cupertino, CA', 'USA'),
('Armonk, NY', 'USA'),
('Seattle, WA', 'USA'),
('Detroit, MI', 'USA'),
('Tokyo', 'Japan'),
('Hangzhou, Zhejiang', 'China'),
('Paris', 'France'),
('Suwon, Gyeonggi-do', 'South Korea'),
('Bentonville, Arkansas', 'USA')
"""
cur.execute(insert_records_locations)


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=my_db) at 0x27c007d2720>

In [24]:
#1. What are the first names of the CEOs of the companies headquartered in South Korea?
query_cmd = '''
    SELECT ceos.first_name
    FROM ceos
    INNER JOIN companies ON ceos.ceo_id = companies.ceo_id
    INNER JOIN locations ON companies.headquarters_city = locations.city
    WHERE locations.country = 'South Korea'
'''
cur.execute(query_cmd)
print("The number of rows: ", cur.rowcount)

row = cur.fetchone()
while row is not None:
    print("First names of the CEOs of the companies headquartered in South Korea:", row[0])
    row = cur.fetchone()

The number of rows:  1
First names of the CEOs of the companies headquartered in South Korea: Lee


In [25]:
#2.Which city hosts the headquarters of the company whose CEO has a last name Yoshida?
query_cmd = '''
    SELECT companies.headquarters_city
    FROM companies
    INNER JOIN ceos ON companies.ceo_id = ceos.ceo_id
    WHERE ceos.last_name = 'Yoshida'
'''
cur.execute(query_cmd)
print("The number of rows: ", cur.rowcount)

row = cur.fetchone()
while row is not None:
    print("City that hosts the headquarter of the company whose CEO has a last name Yoshida:", row[0])
    row = cur.fetchone()

The number of rows:  2
City that hosts the headquarter of the company whose CEO has a last name Yoshida: Tokyo, Japan
City that hosts the headquarter of the company whose CEO has a last name Yoshida: Tokyo


In [18]:
#3. What are the founding years of the companies whose CEOs were born between 1960 and 1969?
query_cmd = '''
    SELECT companies.founding_year
    FROM ceos
    INNER JOIN companies ON ceos.ceo_id = companies.ceo_id
    WHERE ceos.year_of_birth BETWEEN 1960 AND 1969
'''
cur.execute(query_cmd)
print("The number of rows: ", cur.rowcount)

row = cur.fetchone()
while row is not None:
    print("Founding years of the companies whose CEOs were born between 1960 and 1969:", row[0])
    row = cur.fetchone()

The number of rows:  7
Founding years of the companies whose CEOs were born between 1960 and 1969: 1976
Founding years of the companies whose CEOs were born between 1960 and 1969: 1911
Founding years of the companies whose CEOs were born between 1960 and 1969: 1994
Founding years of the companies whose CEOs were born between 1960 and 1969: 1908
Founding years of the companies whose CEOs were born between 1960 and 1969: 1963
Founding years of the companies whose CEOs were born between 1960 and 1969: 1969
Founding years of the companies whose CEOs were born between 1960 and 1969: 1962


In [20]:
#4.How long has the CEO of the company founded in 1908 been serving in their position?
query_cmd = '''
    SELECT 2024 - ceos.start_year AS years_served
    FROM ceos
    INNER JOIN companies ON ceos.ceo_id = companies.ceo_id
    WHERE companies.founding_year = 1908
'''
cur.execute(query_cmd)
print("The number of rows: ", cur.rowcount)

row = cur.fetchone()
while row is not None:
    print("The CEO of the company founded in 1908 has been serving in their position for", row[0],"years.")
    row = cur.fetchone()

The number of rows:  1
The CEO of the company founded in 1908 has been serving in their position for 10 years.


In [29]:
#5.What is the age of the CEO of the company based in Hangzhou? 
query_cmd = '''
    SELECT 2024 - ceos.year_of_birth AS age
    FROM ceos
    INNER JOIN companies ON ceos.ceo_id = companies.ceo_id
    INNER JOIN locations ON companies.headquarters_city = LOCATIONS.city
    WHERE locations.city = 'Hangzhou, Zhejiang'
'''
cur.execute(query_cmd)
print("The number of rows: ", cur.rowcount)

row = cur.fetchone()
while row is not None:
    print("The age of the CEO of the company based in Hangzhou is:", row[0])
    row = cur.fetchone()

The number of rows:  1
The age of the CEO of the company based in Hangzhou is: 52
